## Factorizacion de Matrices

In [1]:
### Ahora en Keras

from __future__ import print_function, division
from builtins import range, input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import tensorflow

from tensorflow import keras

In [2]:
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

In [3]:
df = pd.read_csv('rating.csv')

In [4]:
df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [5]:
user = df['userId'].value_counts().index
map = {k:i for i, k in enumerate(user)}
df['userId'] = df['userId'].map(map)

In [6]:
mov = df['movieId'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
df['movieId'] = df['movieId'].map(map)

In [7]:
N = df['userId'].max()
M = df['movieId'].max()

df.drop('timestamp', axis = 1, inplace = True)

N,M



(270895, 45114)

In [8]:
n = 1
m = 1

from collections import Counter

In [9]:
ucount = Counter(df['userId'])
dft = pd.DataFrame(ucount.most_common(n))
dft_list = list(dft[1])
ucount[dft_list[0]]

322

In [10]:
mcount = Counter(df['movieId'])
dft_2 = pd.DataFrame(mcount.most_common(m))
dft_2_list = list(dft_2[1])
mcount[dft_2_list[0]]

0

In [15]:
mcount[91921]

0

In [12]:
mcount.most_common(5)

[(0, 91921), (1, 91082), (2, 87901), (3, 84078), (4, 77960)]

In [13]:
ucount = Counter(df['userId'])
mcount = Counter(df['movieId'])

uid = [u for u, c in ucount.most_common(n)]
mid = [u for u, c in mcount.most_common(m)]

In [14]:
asd

NameError: name 'asd' is not defined

In [ ]:
uid

In [ ]:
mcount

In [ ]:
newdf = df[df['userId'].isin(uid) & df['movieId'].isin(mid)]
newdf.head()

In [ ]:
newdf

In [ ]:
N = newdf['userId'].max()
M = newdf['movieId'].max()

user = newdf['userId'].value_counts().index
map = {k:i for i, k in enumerate(user)}
newdf['userId'] = newdf['userId'].map(map)


In [ ]:
mov = newdf['movieId'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
newdf['movieId'] = newdf['movieId'].map(map)

In [ ]:
newdf = shuffle(newdf)
cutoff = int(0.8*len(newdf))
cutoff

In [ ]:
train = newdf.iloc[: cutoff]
test = newdf.iloc[cutoff:]

In [ ]:
K = 10
mu = newdf['rating'].mean()
epochs = 25
reg = 0.
N = 1000
M = 800

# KERAS

In [ ]:
u = Input(shape=(1,))
m = Input(shape=(1,))

u_embed = Embedding(N, K, embeddings_regularizer = l2(reg))(u) # size (N,1,K)
m_embed = Embedding(M, K, embeddings_regularizer = l2(reg))(m) # size (M,1,K)

In [ ]:
u_bias = Embedding(N, 1, embeddings_regularizer = l2(reg))(u)
m_bias = Embedding(M, 1, embeddings_regularizer = l2(reg))(m)

x = Dot(axes = 2)([u_embed, m_embed])

x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # N,1

model = Model(inputs = (u, m),
             outputs = x)

model.compile(loss = 'mse',
             optimizer = Adam(learning_rate = 0.01),
             metrics = ['mse'])

In [ ]:
r = model.fit(x = [train['userId'].values, train['movieId'].values],
                 y = train['rating'].values - mu,
                 epochs = epochs,
                 batch_size = 256,
                 validation_data = ([test['userId'].values, test['movieId'].values],
                 test['rating'].values - mu)
                 )

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mse'], label="train mse")
plt.plot(r.history['val_mse'], label="test mse")
plt.legend()
plt.show()

